In [1]:
from typing import Tuple
import numpy as np

import mujoco
from mujoco import viewer

from dm_control import composer, mjcf
from dm_robotics.moma.models.arenas import empty
from dm_robotics.moma import robot

from hydra import compose, initialize
from hydra.utils import instantiate


# load different robot configurations
initialize(version_base=None, config_path="./config", job_name="default_config")
POSITION_CONFIG = compose(config_name="controller_tuning", overrides=["robots=default"])
VELOCITY_CONFIG = compose(config_name="controller_tuning", overrides=["robots=velocity"])
MOTOR_CONFIG = compose(config_name="controller_tuning", overrides=["robots=motor"])

# For now assign default cfg
cfg = MOTOR_CONFIG

/home/peter/.cache/pypoetry/virtualenvs/mujoco-controllers-OtSPoWZf-py3.10/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'controller_tuning': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


## Load Scene

In [2]:
def build_arena(name: str) -> composer.Arena:
    """Build a MuJoCo arena."""
    arena = empty.Arena(name=name)
    arena.mjcf_model.option.timestep = 0.0001
    arena.mjcf_model.option.gravity = (0.0, 0.0, -9.8)
    arena.mjcf_model.size.nconmax = 1000
    arena.mjcf_model.size.njmax = 2000
    arena.mjcf_model.visual.__getattr__("global").offheight = 640
    arena.mjcf_model.visual.__getattr__("global").offwidth = 640
    arena.mjcf_model.visual.map.znear = 0.0005
    return arena


def add_robot_and_gripper(arena: composer.Arena, arm, gripper) -> Tuple[composer.Entity, composer.Entity]:
    """Add a robot and gripper to the arena."""
    # attach the gripper to the robot
    robot.standard_compose(arm=arm, gripper=gripper)

    # define robot base site
    robot_base_site = arena.mjcf_model.worldbody.add(
        "site",
        name="robot_base",
        pos=(0.0, 0.0, 0.0),
    )

    # add the robot and gripper to the arena
    arena.attach(arm, robot_base_site)

    return arm, gripper

# build the base arena
arena = build_arena("base_scene")

# add robot arm and gripper to the arena
arm = instantiate(cfg.robots.arm)
gripper = instantiate(cfg.robots.gripper)
arm, gripper = add_robot_and_gripper(arena, arm, gripper)

physics = mjcf.Physics.from_mjcf_model(arena.mjcf_model)

# set the default joint positions
    # <joint name="panda_joint1" value="0"/>
    # <joint name="panda_joint2" value="-0.785"/>
    # <joint name="panda_joint3" value="0"/>
    # <joint name="panda_joint4" value="-2.356"/>
    # <joint name="panda_joint5" value="0"/>
    # <joint name="panda_joint6" value="1.571"/>
    # <joint name="panda_joint7" value="0.785"/>
physics.data.qpos[:7] = np.array([0, -0.785, 0, -2.356, 0, 1.571, 0.785])

# launch passive viewer
#viewer.launch_passive(physics.model._model, physics.data._data)

Adding actuator for joint: joint1
Adding actuator for joint: joint2
Adding actuator for joint: joint3
Adding actuator for joint: joint4
Adding actuator for joint: joint5
Adding actuator for joint: joint6
Adding actuator for joint: joint7
Adding sensor for joint: pos_joint1
Adding sensor for joint: pos_joint2
Adding sensor for joint: pos_joint3
Adding sensor for joint: pos_joint4
Adding sensor for joint: pos_joint5
Adding sensor for joint: pos_joint6
Adding sensor for joint: pos_joint7


In [3]:
viewer.launch(physics.model._model, physics.data._data)